---

# KDDCUP99

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn

from time import time
from tqdm import tqdm

from sklearn.metrics import roc_auc_score, average_precision_score,auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, auc, precision_recall_curve
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import OneClassSVM as OCSVM

from sklearn.utils import shuffle

import pandas as pd
import os
import sys

seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
print('[INFO] -> Using Device: ', device)

from load_data import get_KDDCUP99_INV as get_data

[INFO] -> Using Device:  cuda


In [2]:
batch_size = 1024

WORK_SPACE = '../Dataset'
FILE = 'kddcup.data_10_percent_corrected'

SPACE_MODELS = './models'
SPACE_AUC = './auc'
SPACE_AUPRC = './auprc'

for n in (SPACE_MODELS, SPACE_AUC, SPACE_AUPRC):
    if not os.path.exists(n):
        os.mkdir(n)

DATASET = 'KDDCUP99'

PATH = os.path.join(WORK_SPACE, FILE)

In [3]:
dataset = get_data(PATH, seed, show=True)

396743 normal records, 97278 anormal records
We use 48640 anomalous records
-----------------------------------------------------------------------------------------
There are 274006 records in training set
Training set is composed by 274006 normal records and 0 abnormal records
-----------------------------------------------------------------------------------------
There are 43701 records in validation set
Validation set is composed by 31540 normal records and 12161 abnormal records
-----------------------------------------------------------------------------------------
There are 127676 records in test set
Test set is composed by 91197 normal records and 36479 abnormal records


## Model

In [4]:
def meanAUC_PRAUC(auc_list, pr_list, name):
    print('AUC:')
    print(auc_list)
    print('**************')
    print('PR AUC:')
    print(pr_list)
    print('**************')
    
    
    AUC_Frame = pd.DataFrame(auc_list, columns = [name])
    PR_AUC_Frame = pd.DataFrame(pr_list, columns = [name])
    
    AUC_Frame.to_csv(f'./auc/{name}.csv', index=False)
    PR_AUC_Frame.to_csv(f'./auc_PR/{name}.csv', index=False)
    
    AUC_Frame = list(AUC_Frame[name])

    N = len(AUC_Frame)
    mean_auc = np.mean(AUC_Frame)
    std_auc = np.std(AUC_Frame)
    std_error = std_auc / (np.sqrt(N))

    ci = 1.96 * std_error
    lower_bound = mean_auc - ci
    upper_bound = mean_auc + ci
    
    print('AUC')
    print(f'{mean_auc:.2f} +/- {ci:.2f}')
    print(f'95% confidence level, average auc would be between {lower_bound:.2f} and {upper_bound:.2f}')
    print('**************')
    
    PR_AUC_Frame = list(PR_AUC_Frame[name])

    N = len(PR_AUC_Frame)
    mean_auc = np.mean(PR_AUC_Frame)
    std_auc = np.std(PR_AUC_Frame)
    std_error = std_auc / (np.sqrt(N))

    ci = 1.96 * std_error
    lower_bound = mean_auc - ci
    upper_bound = mean_auc + ci
    
    print('PR AUC')
    print(f'{mean_auc:.2f} +/- {ci:.2f}')
    print(f'95% confidence level, average auc would be between {lower_bound:.2f} and {upper_bound:.2f}')

def plot_pr_curve(precision, recall):
    plt.figure()
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()

def pr_auc(y_test, y_pred):
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    auc_score = auc(recall, precision)
    print(f'PR AUC: {auc_score:.2f}')
    plot_pr_curve(precision, recall)
    return auc_score

#### Mean AUC and standard deviation after seven runs

In [ ]:
n_runs = 20
auc_list = []
num_epochs = 50
seed = 42
pr_list = []

for i in range(n_runs):
    print(f'Iteration: {i+1}')
    dataset = get_data(PATH, seed*(i+1), show=False)
    
    dataset['y_test'].replace({0:-1  inplace = True)
    
    ocsvm = OCSVM(gamma = 'auto', verbose = 2)
    ocsvm.fit(dataset['x_train'])
    y_pred = ocsvm.score_samples(dataset['x_test'])
    
    y_true = dataset['y_test']
    y_true = 1-(y_true+1)/2
    y_scores = -y_pred
    
    _auc = roc_auc_score(y_true, y_scores)
    pr = pr_auc(y_true, y_scores)
    print(f'AUC: {_auc:.2f}')
    
    auc_list.append(_auc)
    pr_list.append(pr)

In [ ]:
meanAUC_PRAUC(auc_list, pr_list, 'OCSVM_KDDCUP99_INV')

---